In [1]:
%autosave 0

Autosave disabled


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from pydataset import data
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_regression, RFE #feature selection objects
from sklearn.linear_model import LinearRegression

In [3]:
df = data('swiss')
df.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [4]:
# normally you would do the following on train

Lets look at the range of values in our dataset to see if we need to scale!

In [5]:
df.describe()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
count,47.000000,47.000000,47.000000,47.000000,47.00000,47.000000
mean,70.142553,50.659574,16.489362,10.978723,41.14383,19.942553
std,12.491697,22.711218,7.977883,9.615407,41.70485,2.912697
min,35.000000,1.200000,3.000000,1.000000,2.15000,10.800000
25%,64.700000,35.900000,12.000000,6.000000,5.19500,18.150000
50%,70.400000,54.100000,16.000000,8.000000,15.14000,20.000000
75%,78.450000,67.650000,22.000000,12.000000,93.12500,21.700000
max,92.500000,89.700000,37.000000,53.000000,100.00000,26.600000


My fields have different scales, it is wise to scale before continuing!

In [6]:
df.columns = df.columns.str.lower()
df.rename(columns={'infant.mortality': 'infant_mortality'}, inplace=True)
df.head(2)

,fertility,agriculture,examination,education,catholic,infant_mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2


In [7]:
mms = MinMaxScaler()

to_scale = df.drop(columns=['fertility']).columns

df[to_scale] = mms.fit_transform(df[to_scale])

df.head()

,fertility,agriculture,examination,education,catholic,infant_mortality
Courtelary,80.2,0.178531,0.352941,0.211538,0.079816,0.721519
Delemont,83.1,0.496045,0.088235,0.153846,0.845069,0.721519
Franches-Mnt,92.5,0.435028,0.058824,0.076923,0.932550,0.594937
Moutier,85.8,0.398870,0.264706,0.115385,0.323148,0.601266
Neuveville,76.9,0.477966,0.411765,0.269231,0.030761,0.620253


Let's break our data into X and y subsets.

We will move forward with SelectKBest. This technique uses a statistical test to determine how useful features may be.

In [8]:
X = df.drop(columns=['fertility'])
y = df.fertility

In [9]:
skb = SelectKBest(f_regression, k = 3)

skb.fit(X, y)

SelectKBest(k=3, score_func=<function f_regression at 0x15d91df30>)

In [10]:
skb_mask = skb.get_support()
X.columns[skb_mask]

Index(['examination', 'education', 'catholic'], dtype='object')

According to SelectKBest, Examination, Education, and Catholic will have the strongest relationshiop with my target variable.

Now let's do RFE. With RFE, we need a model object that will be used to evaluate the predictive power of our features. We'll use a simple linear regression model to do this.

In [11]:
lm = LinearRegression()

rfe = RFE(lm, n_features_to_select=3)

rfe.fit(X, y)

RFE(estimator=LinearRegression(), n_features_to_select=3)

In [12]:
rfe_mask = rfe.get_support()
X.columns[rfe_mask]

Index(['agriculture', 'education', 'infant_mortality'], dtype='object')

Our RFE object says that agriculture, education, and infant_mortality are the top performing features from our dataset.

How does that compare to the features selected by SelectKBest?

In [13]:
X.columns[skb_mask]

Index(['examination', 'education', 'catholic'], dtype='object')

SelectKBest and RFE returned different results! This is ok! We used two different techniques to determine feature importance.